In [5]:
#all!git clone https://github.com/JonWinkelman/jw_utils.git
#abs!git clone https://github.com/JonWinkelman/orthofinder_utils.git
import os
import pandas as pd
from jw_utils import parse_fasta as pfa
from jw_utils import hmmer_utils as hu
from jw_utils import alignment_utils2 as au
from jw_utils import itol_annotations as ia
from jw_utils import ncbi_utils as nu
from jw_utils import make_bac120_tree as mbt
from jw_utils import reciprocal_blast as rb

from Bio import Phylo
from Bio import SeqIO
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import plotly.io as pio
import random

import subprocess
from concurrent.futures import ThreadPoolExecutor

from pathlib import Path


In [ ]:
## Reciprocal blast

In [12]:
### File paths
DATA_DIR = Path('./data')
RESULTS_DIR = Path('./results')
PROTEOME_DIR = Path('./data/Proteomes/')
NCBI_SUMMARIES_FP = 'data/summaries.json'
BLAST_DB = DATA_DIR / 'blast_db'
REFERENCE_PROTEOME_ACC='GCA_000014625.1'
REFERENCE_PROTEOMEOM_FP = PROTEOME_DIR / f'{REFERENCE_PROTEOME_ACC}.faa'

In [10]:
df_fp = 'data/GCA_000014625.1__v__GCF_000014625.1_full.csv'
gene_prot_d = pd.read_csv(df_fp).set_index('GCA_gene_ID')['GCA_protein_ID'].to_dict()
gene_prot_d['PA14_20480']

'ABJ12623.1'

In [14]:
def run_rblast_on_multiple_genes(name_prot_d, reference_proteome_acc,
                                 reference_proteome_fp, 
                                 par_dir, proteome_dir, blast_db,
                                 make_blast_db=True):
    """"""
    results = {}
    for i, (name, prot_id) in enumerate(name_prot_d.items()):

        
        
        prot_seq = pfa.get_seq_dict(reference_proteome_fp)[prot_id]
        full_prot_id = f'{reference_proteome_acc}_{prot_id}'
        recip_blast_results_dir = Path(par_dir) / f"rblast_results_{full_prot_id}"
        recip_blast_results_dir.mkdir(exist_ok=True, parents=True)
        query_fp = recip_blast_results_dir / f'{full_prot_id}.faa'
        pfa.write_to_fasta({full_prot_id:prot_seq}, query_fp)
        if not Path(blast_db).exists():
            rb.make_blastp_databases(proteome_dir, blast_db)
        
        
        print(f"name: {name},prot_id: {prot_id}, query_fp: {query_fp}, recip_blast_results_dir: {recip_blast_results_dir}")
        df = rb.full_rblast(proteome_dir, str(query_fp), prot_id, reference_proteome_fp, 
                   blast_db_name=blast_db, recip_blast_results_dir=recip_blast_results_dir,
                        make_blast_db=make_blast_db)
    results[name] = df
    return results

query_gene_id_d = {'DimA':'PA14_20480','bphP':'PA14_10700', 'algB':'PA14_72380', 'algW':'PA14_57760'}
name_prot_d = {name:gene_prot_d[gene_id] for name, gene_id in query_gene_id_d.items()}

run_rblast_on_multiple_genes(name_prot_d, reference_proteome_acc=REFERENCE_PROTEOME_ACC,
                                 reference_proteome_fp = REFERENCE_PROTEOMEOM_FP, 
                                 par_dir = RESULTS_DIR, proteome_dir = PROTEOME_DIR,
                            blast_db=BLAST_DB, make_blast_db=False)



Building a new DB, current time: 11/04/2025 13:43:30
New DB name:   /Users/jonathanwinkelman/Trestle/Mukherjee_lab/DimA_manuscript/data/blast_db/GCF_001721805.1/GCF_001721805.1_db
New DB title:  data/Proteomes/GCF_001721805.1.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 6018 sequences in 0.0492198 seconds.


BLAST database created successfully of data/Proteomes/GCF_001721805.1.faa at data/blast_db/GCF_001721805.1/GCF_001721805.1_db (type: prot)


Building a new DB, current time: 11/04/2025 13:43:30
New DB name:   /Users/jonathanwinkelman/Trestle/Mukherjee_lab/DimA_manuscript/data/blast_db/GCA_902497625.1/GCA_902497625.1_db
New DB title:  data/Proteomes/GCA_902497625.1.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 5024 sequences in 0.0284858 seconds.


BLAST database created successfully of data/Proteomes/GCA_902497625.1.faa at data/blast_db/GCA_902497625.1/GCA_90

BLAST options error: File data/Proteomes/.DS_Store.faa does not exist


Adding sequences from FASTA; added 5313 sequences in 0.0307651 seconds.


BLAST database created successfully of data/Proteomes/GCF_014268455.2.faa at data/blast_db/GCF_014268455.2/GCF_014268455.2_db (type: prot)


Building a new DB, current time: 11/04/2025 13:43:33
New DB name:   /Users/jonathanwinkelman/Trestle/Mukherjee_lab/DimA_manuscript/data/blast_db/GCF_022569895.1/GCF_022569895.1_db
New DB title:  data/Proteomes/GCF_022569895.1.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 5710 sequences in 0.0322399 seconds.


BLAST database created successfully of data/Proteomes/GCF_022569895.1.faa at data/blast_db/GCF_022569895.1/GCF_022569895.1_db (type: prot)


Building a new DB, current time: 11/04/2025 13:43:33
New DB name:   /Users/jonathanwinkelman/Trestle/Mukherjee_lab/DimA_manuscript/data/blast_db/GCF_003369775.1/GCF_003369775.1_db
New DB title:  data/Proteomes/GCF_003369775.1.faa
Sequence type: Protein
Keep MBits: T
Maxim

{'algW':                 forward_query         f_bbhit     r_bbhit  recip_best_hit
 target_proteome                                                          
 GCF_000510285.1    ABJ13714.1  WP_015271734.1  ABJ13714.1            True
 GCF_001721825.1    ABJ13714.1  WP_003134756.1  ABJ13714.1            True
 GCF_001449085.1    ABJ13714.1  WP_003205476.1  ABJ13714.1            True
 GCF_020424105.1    ABJ13714.1  WP_025806436.1  ABJ13714.1            True
 GCF_003850385.1    ABJ13714.1  WP_025806436.1  ABJ13714.1            True
 ...                       ...             ...         ...             ...
 GCA_000014625.1    ABJ13714.1      ABJ13714.1  ABJ13714.1            True
 GCF_019719375.1    ABJ13714.1  WP_011332484.1  ABJ13714.1            True
 GCF_002205335.1    ABJ13714.1  WP_003098831.1  ABJ13714.1            True
 GCF_001605965.1    ABJ13714.1  WP_016770590.1  ABJ13714.1            True
 GCF_900625045.1    ABJ13714.1  WP_010565606.1  ABJ13714.1            True
 
 [331 rows x 4 

In [19]:
recip_blast_results_dirs = {Path('./results/rblast_results_GCA_000014625.1_ABJ12623.1/'),
                            Path('./results/rblast_results_GCA_000014625.1_ABJ13387.1/'),
                            Path('./results/rblast_results_GCA_000014625.1_ABJ13714.1/'),
                            Path('./results/rblast_results_GCA_000014625.1_ABJ14869.1/'),
                           }

In [25]:
d = {}
for name, prot_id in name_prot_d.items():
    rblast_results_fp = Path(f'./results/rblast_results_{REFERENCE_PROTEOME_ACC}_{prot_id}/recip_blast_results.csv')
    blast_results_df = pd.read_csv(rblast_results_fp)
    rblast_results_df = blast_results_df.set_index('target_proteome').join(nu.make_summary_df(NCBI_SUMMARIES_FP))
    rblast_results_df.sort_values('recip_best_hit', ascending=False).to_csv(rblast_results_fp.parent / 'recip_blast_results_orgnames.csv')
    d[name] = rblast_results_df



In [27]:
dfs = []
for prot_id, df in d.items():
    org_df = df[['organism_name', 'tax_id']]
    df, prot_id  
    df = df[df['recip_best_hit'] ]
    df=df[['f_bbhit']]
    df = df.rename(columns={'f_bbhit':f'putatative_{prot_id}'})
    dfs.append(df)
df = pd.concat(dfs, axis=1).join(org_df)
df.to_csv('./results/rblast_hits_combined.csv')

In [28]:
df

,putatative_DimA,putatative_bphP,putatative_algB,putatative_algW,organism_name,tax_id
target_proteome,,,,,,
GCF_001721825.1,WP_023111506.1,WP_003118199.1,WP_003096964.1,WP_003134756.1,Pseudomonas aeruginosa,287
GCF_002208645.1,WP_003113127.1,WP_003121929.1,WP_003102840.1,WP_003134756.1,Pseudomonas aeruginosa,287
GCF_021249245.1,WP_003113127.1,WP_003454244.1,WP_003102840.1,WP_003098831.1,Pseudomonas aeruginosa,287
GCF_000006765.1,NP_252059.1,NP_252806.1,NP_254170.1,NP_253136.1,Pseudomonas aeruginosa PAO1,208964
GCF_024760445.1,WP_003113127.1,WP_003101278.1,WP_003102840.1,WP_003098831.1,Pseudomonas aeruginosa,287
...,...,...,...,...,...,...
GCF_013200955.1,NaN,NaN,NaN,WP_173209328.1,Aquipseudomonas campi,2731681
GCF_002362295.1,NaN,NaN,NaN,WP_084900066.1,Burkholderia ubonensis subsp. mesacidophila,265293
GCF_002941105.1,NaN,NaN,NaN,WP_104728230.1,Ectopseudomonas oleovorans,301
